In [6]:
import sys
import os

# Add the path to the aqi package
aqi_path = os.path.abspath('../../../airflow')
sys.path.append(aqi_path)

from plugins.hooks.aqi import fetch_data
from dags.etl.aqi import validate_data, parse_data, clean_data, push_to_db

In [7]:
aqi_data = fetch_data("6814")
print(aqi_data['status'])
print(len(aqi_data['data']))

<Response [200]>
{'status': 'ok', 'data': {'aqi': 122, 'idx': 6814, 'attributions': [{'url': 'http://aqmthai.com/', 'name': 'Division of Air Quality Data, Air Quality and Noise Management Bureau, Pollution Control Department.', 'logo': 'Thailand-PCD.png'}, {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}], 'city': {'geo': [13.5502631, 100.2647676], 'name': 'Provincial Administrative Organization, Samut Sakhon, Thailand (องค์การบริหารส่วนจังหวัดสมุทรสาคร)', 'url': 'https://aqicn.org/city/thailand/samut-sakhon/provincial-administrative-organization', 'location': ''}, 'dominentpol': 'pm25', 'iaqi': {'dew': {'v': -7.6}, 'h': {'v': 11}, 'no2': {'v': 7.5}, 'o3': {'v': 2.8}, 'p': {'v': 1018}, 'pm10': {'v': 33}, 'pm25': {'v': 122}, 'r': {'v': 99.9}, 'so2': {'v': 55.4}, 't': {'v': 24.9}, 'w': {'v': 1}}, 'time': {'s': '2025-02-25 09:00:00', 'tz': '+07:00', 'v': 1740474000, 'iso': '2025-02-25T09:00:00+07:00'}, 'forecast': {'daily': {'pm10': [{'avg': 49, 'day': '2025-02-23'

In [8]:
validate_data(aqi_data)

True

In [9]:
print(aqi_data["data"].get("idx"))

parse_aqi_data = parse_data(aqi_data)
print(parse_aqi_data)

6814
    uid                                            address  aqi  dew   h   o3  \
0  6814  Provincial Administrative Organization, Samut ...  122 -7.6  11  2.8   

      p  pm10  pm25     r     t  w                 timestamp  \
0  1018    33   122  99.9  24.9  1 2025-02-25 09:00:00+07:00   

                                            forecast  
0  {"pm10": [{"avg": 49, "day": "2025-02-23", "ma...  


In [5]:
clean_aqi_data = clean_data(parse_aqi_data)
print(clean_aqi_data)

    idx                                            address  aqi  dew     h  \
0  6814  Provincial Administrative Organization, Samut ...   70 -7.9  10.1   

    o3       p  pm10  pm25     r   t  w                 timestamp  \
0  2.8  1016.7    33    70  99.8  26  1 2025-02-24 23:00:00+07:00   

                                            forecast  
0  {"o3": [{"avg": 13, "day": "2025-02-22", "max"...  


In [11]:
import psycopg2

# Database connection parameters
db_params = {
    'dbname': 'air_data',
    'user': 'admin',
    'password': 'adminpassword',
    'host': 'localhost',
    'port': '54329'
}

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)

# Push the cleaned AQI data to the database
push_data = push_to_db(conn, clean_aqi_data)

# Close the connection
conn.close()